In [1]:
import pandas as pd
df_wells = pd.read_excel("H:\Sub_New_Model\monthly_time_series_OK2.xlsx")

In [2]:
df_wells

,ID_Well,W1,W2,W3,W4,W5,W6,W7,W8,W9,...,W76,W77,W78,W79,W80,W81,W82,W83,W84,W85
0,Latitude,36.098206,36.110136,36.149156,36.160917,36.173444,36.191585,36.204182,36.231061,36.239237,...,36.842208,36.857473,36.858884,36.874898,36.881811,36.892981,36.927662,36.935502,36.947604,36.970626
1,Longitude,59.761354,59.801812,59.828758,59.747949,59.674010,59.667589,59.752442,59.894612,59.612835,...,58.844568,58.755551,58.911753,58.668451,58.820747,58.771076,58.701586,58.620536,58.753912,58.699273
2,1992-09,89.050000,50.130000,26.670000,69.320000,95.000000,NaN,NaN,17.130000,68.740000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992-10,89.030000,50.250000,26.710000,69.480000,94.850000,NaN,NaN,17.340000,68.650000,...,48.250000,77.990000,59.140000,38.930000,70.910000,54.800000,13.600000,53.450000,49.700000,15.450000
4,1992-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.800000,78.080000,59.260000,38.940000,70.940000,54.810000,13.610000,53.430000,49.720000,15.550000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,2023-05,129.430000,NaN,NaN,112.210000,NaN,104.480000,90.530000,12.420000,NaN,...,90.210000,NaN,89.040000,42.080000,97.830000,77.900000,NaN,NaN,NaN,44.180000
365,2023-06,129.610000,NaN,NaN,112.410000,NaN,104.440000,90.570000,12.760000,NaN,...,90.590000,NaN,89.080000,42.110000,97.880000,78.130000,NaN,NaN,NaN,44.660000
366,2023-07,129.710000,NaN,NaN,112.520000,NaN,104.400000,90.630000,13.010000,NaN,...,90.800000,NaN,89.180000,42.180000,97.970000,78.250000,NaN,NaN,NaN,44.990000
367,2023-08,129.910000,NaN,NaN,112.780000,NaN,104.380000,90.590000,13.410000,NaN,...,91.020000,NaN,89.250000,42.290000,98.030000,78.300000,NaN,NaN,NaN,45.160000


In [6]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import os
from sklearn.linear_model import LinearRegression

# Function to calculate Haversine distance between two points (in kilometers)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Define the output folder path
output_folder = r"H:\Sub_New_Model\Groundwater_Clusters_Per_Well258"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the data from the specified Excel file
df_wells = pd.read_excel(r"H:\Sub_New_Model\monthly_time_series_OK2.xlsx")

# Extract latitude and longitude
latitudes = df_wells.loc[df_wells['ID_Well'] == 'Latitude'].iloc[0, 1:].astype(float)
longitudes = df_wells.loc[df_wells['ID_Well'] == 'Longitude'].iloc[0, 1:].astype(float)

# Extract groundwater depth data (from row 2 onwards)
depth_data = df_wells.loc[2:, 'W1':].astype(float)
depth_data.index = df_wells.loc[2:, 'ID_Well']

# Conditions for distance and correlation
conditions = [
    {'distance': 3, 'correlation': 0.7},
    {'distance': 4, 'correlation': 0.8},
    {'distance': 5, 'correlation': 0.9},
    {'distance': 12, 'correlation': 0.95}
]

# Get all wells
wells = depth_data.columns

# Process each well individually
for target_well in wells:
    # Dictionary to store pairs for this well for each condition
    well_pairs = {i: [] for i in range(len(conditions))}
    
    # Calculate distances and correlations for this well with all others
    for other_well in wells:
        if target_well == other_well:
            continue
        
        # Calculate distance
        lat1, lon1 = latitudes[target_well], longitudes[target_well]
        lat2, lon2 = latitudes[other_well], longitudes[other_well]
        distance = haversine(lat1, lon1, lat2, lon2)
        
        # Calculate correlation (drop NaN pairs)
        series1, series2 = depth_data[target_well], depth_data[other_well]
        valid_pairs = series1.notna() & series2.notna()
        if valid_pairs.sum() < 3:  # Need at least 3 valid points for correlation
            continue
        correlation = series1[valid_pairs].corr(series2[valid_pairs])
        
        if np.isnan(correlation):
            continue
        
        # Check each condition
        for i, cond in enumerate(conditions):
            if distance < cond['distance'] and correlation > cond['correlation']:
                well_pairs[i].append((target_well, other_well, distance, correlation))
    
    # Copy depth data for estimation
    estimated_data = depth_data[target_well].copy()
    is_estimated = pd.Series(False, index=estimated_data.index)
    
    # Estimate missing values using regression
    for time in estimated_data.index:
        if not np.isnan(estimated_data[time]):
            continue  # Skip if data exists
        
        # Collect neighbors with data at this time
        neighbors = []
        for cond_idx, pairs in well_pairs.items():
            for w1, w2, dist, corr in pairs:
                if w1 != target_well:
                    continue
                if not np.isnan(depth_data[w2][time]):
                    neighbors.append((w2, dist, corr, depth_data[w2][time]))
        
        # If no neighbors have data, skip
        if not neighbors:
            continue
        
        # Prepare data for regression: ensure no NaN in training data
        valid_times = depth_data[target_well].notna()
        if valid_times.sum() < 3:  # Need at least 3 valid points for regression
            continue
        
        # If only one neighbor, use simple linear regression
        if len(neighbors) == 1:
            neighbor_well, _, _, neighbor_value = neighbors[0]
            # Filter valid times where both target and neighbor have data
            valid_for_regression = valid_times & depth_data[neighbor_well].notna()
            if valid_for_regression.sum() < 3:  # Need at least 3 valid points
                continue
            X = depth_data[neighbor_well][valid_for_regression].values.reshape(-1, 1)
            y = depth_data[target_well][valid_for_regression].values
            model = LinearRegression()
            model.fit(X, y)
            predicted_value = model.predict([[neighbor_value]])[0]
            estimated_data[time] = predicted_value
            is_estimated[time] = True
        
        # If multiple neighbors, use multiple linear regression
        else:
            neighbor_wells = [w for w, _, _, _ in neighbors]
            neighbor_values = [v for _, _, _, v in neighbors]
            # Find times where target and ALL neighbors have valid data
            valid_for_regression = valid_times.copy()
            for w in neighbor_wells:
                valid_for_regression = valid_for_regression & depth_data[w].notna()
            if valid_for_regression.sum() < 3:  # Need at least 3 valid points
                # Fallback to single regression with best neighbor (highest correlation)
                if neighbors:
                    best_neighbor = max(neighbors, key=lambda x: x[2])  # Max by correlation
                    neighbor_well, _, _, neighbor_value = best_neighbor
                    valid_for_regression = valid_times & depth_data[neighbor_well].notna()
                    if valid_for_regression.sum() < 3:
                        continue
                    X = depth_data[neighbor_well][valid_for_regression].values.reshape(-1, 1)
                    y = depth_data[target_well][valid_for_regression].values
                    model = LinearRegression()
                    model.fit(X, y)
                    predicted_value = model.predict([[neighbor_value]])[0]
                    estimated_data[time] = predicted_value
                    is_estimated[time] = True
                continue
            
            # Prepare X for multiple regression
            X = []
            for w in neighbor_wells:
                X.append(depth_data[w][valid_for_regression].values)
            X = np.array(X).T  # Transpose to shape (n_samples, n_neighbors)
            y = depth_data[target_well][valid_for_regression].values
            model = LinearRegression()
            model.fit(X, y)
            predicted_value = model.predict([neighbor_values])[0]
            estimated_data[time] = predicted_value
            is_estimated[time] = True
    
    # Prepare output for this well
    output_data = []
    for cond_idx, pairs in well_pairs.items():
        if not pairs:
            continue
        
        # Add the target well's data
        target_row = {
            'Well': target_well,
            'Latitude': latitudes[target_well],
            'Longitude': longitudes[target_well],
            'Condition': f"Dist<{conditions[cond_idx]['distance']}km, Corr>{conditions[cond_idx]['correlation']}",
            'Distance_km': '',
            'Correlation': ''
        }
        target_row.update(estimated_data.to_dict())
        target_row['Is_Estimated'] = is_estimated.to_dict()
        output_data.append(target_row)
        
        # Add each neighbor's data vertically below the target well
        for w1, w2, dist, corr in pairs:
            if w1 != target_well:
                continue
            neighbor_row = {
                'Well': w2,
                'Latitude': latitudes[w2],
                'Longitude': longitudes[w2],
                'Condition': f"Dist<{conditions[cond_idx]['distance']}km, Corr>{conditions[cond_idx]['correlation']}",
                'Distance_km': f"{dist:.2f}",
                'Correlation': f"{corr:.3f}"
            }
            neighbor_row.update(depth_data[w2].to_dict())
            neighbor_row['Is_Estimated'] = pd.Series(False, index=depth_data.index).to_dict()
            output_data.append(neighbor_row)
    
    # Create DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Rename columns to match dates
    date_columns = {date: date for date in depth_data.index}
    output_df.rename(columns=date_columns, inplace=True)
    
    # Save to Excel in the specified folder
    output_file = os.path.join(output_folder, f'groundwater_cluster_estimated_{target_well}.xlsx')
    output_df.to_excel(output_file, index=False)
    print(f"Saved cluster and estimated data for well {target_well} to {output_file}")

Saved cluster and estimated data for well W1 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well258\groundwater_cluster_estimated_W1.xlsx
Saved cluster and estimated data for well W2 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well258\groundwater_cluster_estimated_W2.xlsx
Saved cluster and estimated data for well W3 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well258\groundwater_cluster_estimated_W3.xlsx
Saved cluster and estimated data for well W4 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well258\groundwater_cluster_estimated_W4.xlsx
Saved cluster and estimated data for well W5 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well258\groundwater_cluster_estimated_W5.xlsx
Saved cluster and estimated data for well W6 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well258\groundwater_cluster_estimated_W6.xlsx
Saved cluster and estimated data for well W7 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well258\groundwater_cluster_estimated_W7.xlsx
Saved cluster and estimated data for well W8 to 

In [7]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import os
from sklearn.linear_model import LinearRegression

# Function to calculate Haversine distance between two points (in kilometers)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Define the output folder path
output_folder = r"H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the data from the specified Excel file
df_wells = pd.read_excel(r"H:\Sub_New_Model\monthly_time_series_OK2.xlsx")

# Extract latitude and longitude
latitudes = df_wells.loc[df_wells['ID_Well'] == 'Latitude'].iloc[0, 1:].astype(float)
longitudes = df_wells.loc[df_wells['ID_Well'] == 'Longitude'].iloc[0, 1:].astype(float)

# Extract groundwater depth data (from row 2 onwards)
depth_data = df_wells.loc[2:, 'W1':].astype(float)
depth_data.index = df_wells.loc[2:, 'ID_Well']

# Conditions for distance and correlation
conditions = [
    {'distance': 3, 'correlation': 0.7},
    {'distance': 4, 'correlation': 0.8},
    {'distance': 5, 'correlation': 0.9},
    {'distance': 12, 'correlation': 0.95}
]

# Get all wells
wells = depth_data.columns

# Dictionary to store estimated data and estimation status for all wells
all_estimated_data = {}
all_is_estimated = {}

# Process each well individually for estimation
for target_well in wells:
    # Dictionary to store pairs for this well for each condition
    well_pairs = {i: [] for i in range(len(conditions))}
    
    # Calculate distances and correlations for this well with all others
    for other_well in wells:
        if target_well == other_well:
            continue
        
        # Calculate distance
        lat1, lon1 = latitudes[target_well], longitudes[target_well]
        lat2, lon2 = latitudes[other_well], longitudes[other_well]
        distance = haversine(lat1, lon1, lat2, lon2)
        
        # Calculate correlation (drop NaN pairs)
        series1, series2 = depth_data[target_well], depth_data[other_well]
        valid_pairs = series1.notna() & series2.notna()
        if valid_pairs.sum() < 3:  # Need at least 3 valid points for correlation
            continue
        correlation = series1[valid_pairs].corr(series2[valid_pairs])
        
        if np.isnan(correlation):
            continue
        
        # Check each condition
        for i, cond in enumerate(conditions):
            if distance < cond['distance'] and correlation > cond['correlation']:
                well_pairs[i].append((target_well, other_well, distance, correlation))
    
    # Copy depth data for estimation
    estimated_data = depth_data[target_well].copy()
    is_estimated = pd.Series(False, index=estimated_data.index)
    
    # Estimate missing values using regression
    for time in estimated_data.index:
        if not np.isnan(estimated_data[time]):
            continue  # Skip if data exists
        
        # Collect neighbors with data at this time
        neighbors = []
        for cond_idx, pairs in well_pairs.items():
            for w1, w2, dist, corr in pairs:
                if w1 != target_well:
                    continue
                if not np.isnan(depth_data[w2][time]):
                    neighbors.append((w2, dist, corr, depth_data[w2][time]))
        
        # If no neighbors have data, skip
        if not neighbors:
            continue
        
        # Prepare data for regression: ensure no NaN in training data
        valid_times = depth_data[target_well].notna()
        if valid_times.sum() < 3:  # Need at least 3 valid points for regression
            continue
        
        # If only one neighbor, use simple linear regression
        if len(neighbors) == 1:
            neighbor_well, _, _, neighbor_value = neighbors[0]
            # Filter valid times where both target and neighbor have data
            valid_for_regression = valid_times & depth_data[neighbor_well].notna()
            if valid_for_regression.sum() < 3:  # Need at least 3 valid points
                continue
            X = depth_data[neighbor_well][valid_for_regression].values.reshape(-1, 1)
            y = depth_data[target_well][valid_for_regression].values
            model = LinearRegression()
            model.fit(X, y)
            predicted_value = model.predict([[neighbor_value]])[0]
            estimated_data[time] = predicted_value
            is_estimated[time] = True
        
        # If multiple neighbors, use multiple linear regression
        else:
            neighbor_wells = [w for w, _, _, _ in neighbors]
            neighbor_values = [v for _, _, _, v in neighbors]
            # Find times where target and ALL neighbors have valid data
            valid_for_regression = valid_times.copy()
            for w in neighbor_wells:
                valid_for_regression = valid_for_regression & depth_data[w].notna()
            if valid_for_regression.sum() < 3:  # Need at least 3 valid points
                # Fallback to single regression with best neighbor (highest correlation)
                if neighbors:
                    best_neighbor = max(neighbors, key=lambda x: x[2])  # Max by correlation
                    neighbor_well, _, _, neighbor_value = best_neighbor
                    valid_for_regression = valid_times & depth_data[neighbor_well].notna()
                    if valid_for_regression.sum() < 3:
                        continue
                    X = depth_data[neighbor_well][valid_for_regression].values.reshape(-1, 1)
                    y = depth_data[target_well][valid_for_regression].values
                    model = LinearRegression()
                    model.fit(X, y)
                    predicted_value = model.predict([[neighbor_value]])[0]
                    estimated_data[time] = predicted_value
                    is_estimated[time] = True
                continue
            
            # Prepare X for multiple regression
            X = []
            for w in neighbor_wells:
                X.append(depth_data[w][valid_for_regression].values)
            X = np.array(X).T  # Transpose to shape (n_samples, n_neighbors)
            y = depth_data[target_well][valid_for_regression].values
            model = LinearRegression()
            model.fit(X, y)
            predicted_value = model.predict([neighbor_values])[0]
            estimated_data[time] = predicted_value
            is_estimated[time] = True
    
    # Store estimated data and estimation status
    all_estimated_data[target_well] = estimated_data
    all_is_estimated[target_well] = is_estimated

# Prepare combined output for all wells
combined_data = []
# First row: Well names
well_names = {'ID_Well': 'ID_Well'}
well_names.update({w: w for w in wells})
combined_data.append(well_names)

# Second row: Latitude
lat_row = {'ID_Well': 'Latitude'}
lat_row.update(latitudes.to_dict())
combined_data.append(lat_row)

# Third row: Longitude
lon_row = {'ID_Well': 'Longitude'}
lon_row.update(longitudes.to_dict())
combined_data.append(lon_row)

# Add time series data (original and estimated)
for time in depth_data.index:
    row = {'ID_Well': time}
    for w in wells:
        value = all_estimated_data[w][time]
        if np.isnan(value):
            row[w] = np.nan  # Keep NaN if no estimation possible
        else:
            row[w] = value
    combined_data.append(row)

# Create DataFrame for combined data
combined_df = pd.DataFrame(combined_data)

# Save combined data to Excel
combined_output_file = os.path.join(output_folder, 'groundwater_all_wells_estimated.xlsx')
combined_df.to_excel(combined_output_file, index=False)
print(f"Saved combined estimated and original data for all wells to {combined_output_file}")

# Prepare individual output for each well
for target_well in wells:
    output_data = []
    for cond_idx, pairs in well_pairs.items():
        if not pairs:
            continue
        
        # Add the target well's data
        target_row = {
            'Well': target_well,
            'Latitude': latitudes[target_well],
            'Longitude': longitudes[target_well],
            'Condition': f"Dist<{conditions[cond_idx]['distance']}km, Corr>{conditions[cond_idx]['correlation']}",
            'Distance_km': '',
            'Correlation': ''
        }
        target_row.update(all_estimated_data[target_well].to_dict())
        target_row['Is_Estimated'] = all_is_estimated[target_well].to_dict()
        output_data.append(target_row)
        
        # Add each neighbor's data vertically below the target well
        for w1, w2, dist, corr in pairs:
            if w1 != target_well:
                continue
            neighbor_row = {
                'Well': w2,
                'Latitude': latitudes[w2],
                'Longitude': longitudes[w2],
                'Condition': f"Dist<{conditions[cond_idx]['distance']}km, Corr>{conditions[cond_idx]['correlation']}",
                'Distance_km': f"{dist:.2f}",
                'Correlation': f"{corr:.3f}"
            }
            neighbor_row.update(depth_data[w2].to_dict())
            neighbor_row['Is_Estimated'] = pd.Series(False, index=depth_data.index).to_dict()
            output_data.append(neighbor_row)
    
    # Create DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Rename columns to match dates
    date_columns = {date: date for date in depth_data.index}
    output_df.rename(columns=date_columns, inplace=True)
    
    # Save to Excel in the specified folder
    output_file = os.path.join(output_folder, f'groundwater_cluster_estimated_{target_well}.xlsx')
    output_df.to_excel(output_file, index=False)
    print(f"Saved cluster and estimated data for well {target_well} to {output_file}")

Saved combined estimated and original data for all wells to H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_all_wells_estimated.xlsx
Saved cluster and estimated data for well W1 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_cluster_estimated_W1.xlsx
Saved cluster and estimated data for well W2 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_cluster_estimated_W2.xlsx
Saved cluster and estimated data for well W3 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_cluster_estimated_W3.xlsx
Saved cluster and estimated data for well W4 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_cluster_estimated_W4.xlsx
Saved cluster and estimated data for well W5 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_cluster_estimated_W5.xlsx
Saved cluster and estimated data for well W6 to H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_cluster_estima

In [8]:
# Step 1: This script assumes 'groundwater_all_wells_estimated.xlsx' has been generated from the previous step.
# It reads the combined data, fills remaining missing values for each well using the average of the three
# points before and after the missing value, and updates the data iteratively so that estimated values
# can be used in subsequent estimations. The updated data is saved to a new Excel file.

import pandas as pd
import numpy as np
import os

# Define the output folder path
output_folder = r"H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the combined estimated data from the previous step
input_file = r"H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_all_wells_estimated.xlsx"
df = pd.read_excel(input_file)

# Extract well names, latitude, and longitude
well_names = df.loc[df['ID_Well'] == 'ID_Well'].iloc[0, 1:].values
latitudes = df.loc[df['ID_Well'] == 'Latitude'].iloc[0, 1:].astype(float)
longitudes = df.loc[df['ID_Well'] == 'Longitude'].iloc[0, 1:].astype(float)

# Extract time series data (from row 3 onwards)
depth_data = df.loc[3:, 'ID_Well':].copy()
depth_data.index = depth_data['ID_Well']
depth_data = depth_data.iloc[:, 1:].astype(float)  # Convert to float, exclude ID_Well column

# Function to fill missing values using the average of 3 points before and after
def fill_missing_with_window(series):
    updated_series = series.copy()
    for idx in series.index:
        if np.isnan(updated_series[idx]):
            # Get indices for 3 points before and after
            current_pos = series.index.get_loc(idx)
            start_idx = max(0, current_pos - 3)
            end_idx = min(len(series), current_pos + 4)  # +4 because end is exclusive
            window = updated_series.iloc[start_idx:end_idx]
            # Exclude the current index (the NaN value) from the window
            window = window.drop(idx, errors='ignore')
            # Calculate mean of non-NaN values in the window
            if window.notna().sum() > 0:  # Only fill if there are valid values
                updated_series[idx] = window.mean()
    return updated_series

# Iteratively fill missing values for each well
updated_data = depth_data.copy()
max_iterations = 10  # Maximum iterations to ensure convergence
for iteration in range(max_iterations):
    missing_count_before = updated_data.isna().sum().sum()
    for well in well_names:
        updated_data[well] = fill_missing_with_window(updated_data[well])
    missing_count_after = updated_data.isna().sum().sum()
    # If no more missing values are filled, break the loop
    if missing_count_before == missing_count_after:
        break

# Prepare combined output for all wells
combined_data = []
# First row: Well names
well_names_row = {'ID_Well': 'ID_Well'}
well_names_row.update({w: w for w in well_names})
combined_data.append(well_names_row)

# Second row: Latitude
lat_row = {'ID_Well': 'Latitude'}
lat_row.update(latitudes.to_dict())
combined_data.append(lat_row)

# Third row: Longitude
lon_row = {'ID_Well': 'Longitude'}
lon_row.update(longitudes.to_dict())
combined_data.append(lon_row)

# Add updated time series data
for time in updated_data.index:
    row = {'ID_Well': time}
    for w in well_names:
        value = updated_data[w][time]
        row[w] = value  # Keep NaN if still missing
    combined_data.append(row)

# Create DataFrame for combined data
combined_df = pd.DataFrame(combined_data)

# Save updated combined data to Excel
output_file = os.path.join(output_folder, 'groundwater_all_wells_fully_estimated.xlsx')
combined_df.to_excel(output_file, index=False)
print(f"Saved fully updated and estimated data for all wells to {output_file}")

Saved fully updated and estimated data for all wells to H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_all_wells_fully_estimated.xlsx


In [9]:
import pandas as pd
import numpy as np
import os

# Define the output folder path
output_folder = r"H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the combined estimated data from the previous step
combined_file = r"H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_all_wells_estimated.xlsx"
df_combined = pd.read_excel(combined_file)

# Extract latitude and longitude
latitudes = df_combined.loc[df_combined['ID_Well'] == 'Latitude'].iloc[0, 1:].astype(float)
longitudes = df_combined.loc[df_combined['ID_Well'] == 'Longitude'].iloc[0, 1:].astype(float)

# Extract time series data (from row 2 onwards)
depth_data = df_combined.loc[2:, 'W1':].astype(float)
depth_data.index = df_combined.loc[2:, 'ID_Well']

# Get all wells
wells = depth_data.columns

# Copy data for further estimation
updated_data = depth_data.copy()

# Fill remaining missing values using the average of previous and next points
for well in wells:
    series = updated_data[well].copy()
    for idx in series.index:
        if np.isnan(series[idx]):  # Check if value is missing
            # Find previous and next non-NaN values
            prev_idx = series[:idx].last_valid_index()
            next_idx = series[idx:].first_valid_index()
            
            # If both previous and next values exist, compute average
            if prev_idx is not None and next_idx is not None:
                prev_value = series[prev_idx]
                next_value = series[next_idx]
                series[idx] = (prev_value + next_value) / 2
            # If only previous value exists, use it
            elif prev_idx is not None:
                series[idx] = series[prev_idx]
            # If only next value exists, use it
            elif next_idx is not None:
                series[idx] = series[next_idx]
            # If no previous or next value exists, leave as NaN
            # Update the series progressively to use filled values for future estimations
            updated_data[well] = series

# Prepare combined output for all wells
combined_data = []
# First row: Well names
well_names = {'ID_Well': 'ID_Well'}
well_names.update({w: w for w in wells})
combined_data.append(well_names)

# Second row: Latitude
lat_row = {'ID_Well': 'Latitude'}
lat_row.update(latitudes.to_dict())
combined_data.append(lat_row)

# Third row: Longitude
lon_row = {'ID_Well': 'Longitude'}
lon_row.update(longitudes.to_dict())
combined_data.append(lon_row)

# Add updated time series data
for time in updated_data.index:
    row = {'ID_Well': time}
    for w in wells:
        value = updated_data[w][time]
        row[w] = value  # Keep NaN if still missing
    combined_data.append(row)

# Create DataFrame for combined data
combined_df = pd.DataFrame(combined_data)

# Save updated combined data to Excel
combined_output_file = os.path.join(output_folder, 'groundwater_all_wells_estimated_updated.xlsx')
combined_df.to_excel(combined_output_file, index=False)
print(f"Saved updated combined data for all wells to {combined_output_file}")

Saved updated combined data for all wells to H:\Sub_New_Model\Groundwater_Clusters_Per_Well_Estimated\groundwater_all_wells_estimated_updated.xlsx
